### <font color="green">Importing Neccessary Packages and Libraries</font>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import bz2
import os
from dateutil.parser import parse
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
warnings.filterwarnings('ignore')

%matplotlib inline

### <font color="green">Loading Data</font>

In [3]:
def load_data(file):
    data = bz2.BZ2File(file, "r")
    data = pickle.load(data)
    return data

In [4]:
PATH_TRAIN = '../data/raw/act_train.pbz2'
PATH_TEST = '../data/raw/act_test.pbz2'
PATH_PEOPLE = '../data/raw/people.pbz2'
train = load_data(PATH_TRAIN)
test = load_data(PATH_TEST)
people = load_data(PATH_PEOPLE)

In [5]:
train.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [6]:
train.tail()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
2197286,ppl_99994,act2_4668076,2023-06-16,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 418,1
2197287,ppl_99994,act2_4743548,2023-03-30,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1832,1
2197288,ppl_99994,act2_536973,2023-01-19,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1
2197289,ppl_99994,act2_688656,2023-05-02,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 199,1
2197290,ppl_99994,act2_715089,2023-06-15,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,1


In [7]:
train.columns

Index(['people_id', 'activity_id', 'date', 'activity_category', 'char_1',
       'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8',
       'char_9', 'char_10', 'outcome'],
      dtype='object')

In [8]:
row, col = train.shape
print('The length of the train row:', row)
print('The length of the train column:', col)

The length of the train row: 2197291
The length of the train column: 15


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2197291 entries, 0 to 2197290
Data columns (total 15 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   people_id          object
 1   activity_id        object
 2   date               object
 3   activity_category  object
 4   char_1             object
 5   char_2             object
 6   char_3             object
 7   char_4             object
 8   char_5             object
 9   char_6             object
 10  char_7             object
 11  char_8             object
 12  char_9             object
 13  char_10            object
 14  outcome            int64 
dtypes: int64(1), object(14)
memory usage: 251.5+ MB


In [10]:
train.isna().sum() / train.shape[0] # ratio/percentage of the null values for each attribute

people_id            0.000000
activity_id          0.000000
date                 0.000000
activity_category    0.000000
char_1               0.928268
char_2               0.928268
char_3               0.928268
char_4               0.928268
char_5               0.928268
char_6               0.928268
char_7               0.928268
char_8               0.928268
char_9               0.928268
char_10              0.071732
outcome              0.000000
dtype: float64

<p>We can see clearly below the bottom of the series above that the columns with names "char_#" has
about 93% of its values being null.<br> Let's remove them then.</p>

In [11]:
df = train.dropna(axis=1, ignore_index=True)
df.isna().sum() / df.shape[0]

people_id            0.0
activity_id          0.0
date                 0.0
activity_category    0.0
outcome              0.0
dtype: float64

<p>Let's explore the people dataframe and see what we've got there too.</p>

In [12]:
people.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [13]:
people.tail()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
189113,ppl_99987,type 1,group 8600,type 1,2022-04-02,type 4,type 6,type 4,type 3,type 11,...,False,False,False,True,False,True,False,True,True,89
189114,ppl_9999,type 2,group 17304,type 2,2023-02-23,type 6,type 2,type 8,type 3,type 11,...,False,False,False,False,False,False,False,False,False,0
189115,ppl_99992,type 2,group 17304,type 2,2020-06-25,type 5,type 5,type 3,type 4,type 16,...,False,False,False,False,False,False,False,False,False,0
189116,ppl_99994,type 2,group 17764,type 3,2023-01-06,type 2,type 7,type 2,type 1,type 2,...,True,True,True,True,False,True,True,True,True,95
189117,ppl_99997,type 2,group 17304,type 2,2022-03-12,type 40,type 25,type 9,type 3,type 8,...,False,False,False,False,False,False,False,False,False,36


In [14]:
row, col = people.shape
print('The number of rows in train:', row)
print('The number of columns in train:', col)

The number of rows in train: 189118
The number of columns in train: 41


In [15]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189118 entries, 0 to 189117
Data columns (total 41 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   people_id  189118 non-null  object
 1   char_1     189118 non-null  object
 2   group_1    189118 non-null  object
 3   char_2     189118 non-null  object
 4   date       189118 non-null  object
 5   char_3     189118 non-null  object
 6   char_4     189118 non-null  object
 7   char_5     189118 non-null  object
 8   char_6     189118 non-null  object
 9   char_7     189118 non-null  object
 10  char_8     189118 non-null  object
 11  char_9     189118 non-null  object
 12  char_10    189118 non-null  bool  
 13  char_11    189118 non-null  bool  
 14  char_12    189118 non-null  bool  
 15  char_13    189118 non-null  bool  
 16  char_14    189118 non-null  bool  
 17  char_15    189118 non-null  bool  
 18  char_16    189118 non-null  bool  
 19  char_17    189118 non-null  bool  
 20  char

<p>Let's merge the people data with the train data based on the peeple_id as key</p?

In [16]:
new_df = df.merge(people, on=["people_id"], how="inner")

In [17]:
new_df.head()

,people_id,activity_id,date_x,activity_category,outcome,char_1,group_1,char_2,date_y,char_3,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,act2_1734928,2023-08-26,type 4,0,type 2,group 17304,type 2,2021-06-29,type 5,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100,act2_2434093,2022-09-27,type 2,0,type 2,group 17304,type 2,2021-06-29,type 5,...,False,True,True,False,False,True,True,True,False,36
2,ppl_100,act2_3404049,2022-09-27,type 2,0,type 2,group 17304,type 2,2021-06-29,type 5,...,False,True,True,False,False,True,True,True,False,36
3,ppl_100,act2_3651215,2023-08-04,type 2,0,type 2,group 17304,type 2,2021-06-29,type 5,...,False,True,True,False,False,True,True,True,False,36
4,ppl_100,act2_4109017,2023-08-26,type 2,0,type 2,group 17304,type 2,2021-06-29,type 5,...,False,True,True,False,False,True,True,True,False,36


In [18]:
pd.DataFrame(new_df.isna().sum() / new_df.shape[0], columns=["NA_Percentage"])

,NA_Percentage
people_id,0.0
activity_id,0.0
date_x,0.0
activity_category,0.0
outcome,0.0
char_1,0.0
group_1,0.0
char_2,0.0
date_y,0.0
char_3,0.0


### <font color="green">Feature Engineering<font>

In [19]:
new_df[["date_x", "date_y"]].head()

,date_x,date_y
0,2023-08-26,2021-06-29
1,2022-09-27,2021-06-29
2,2022-09-27,2021-06-29
3,2023-08-04,2021-06-29
4,2023-08-26,2021-06-29


In [20]:
new_df = new_df.rename(columns={"date_x": "activity_date", "date_y": "people_date"})
new_df.head()

,people_id,activity_id,activity_date,activity_category,outcome,char_1,group_1,char_2,people_date,char_3,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,act2_1734928,2023-08-26,type 4,0,type 2,group 17304,type 2,2021-06-29,type 5,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100,act2_2434093,2022-09-27,type 2,0,type 2,group 17304,type 2,2021-06-29,type 5,...,False,True,True,False,False,True,True,True,False,36
2,ppl_100,act2_3404049,2022-09-27,type 2,0,type 2,group 17304,type 2,2021-06-29,type 5,...,False,True,True,False,False,True,True,True,False,36
3,ppl_100,act2_3651215,2023-08-04,type 2,0,type 2,group 17304,type 2,2021-06-29,type 5,...,False,True,True,False,False,True,True,True,False,36
4,ppl_100,act2_4109017,2023-08-26,type 2,0,type 2,group 17304,type 2,2021-06-29,type 5,...,False,True,True,False,False,True,True,True,False,36


In [21]:
def extract_year_month_day(df, column_name):
    df[column_name + '_year'] = df[column_name].dt.year
    df[column_name + '_month'] = df[column_name].dt.month
    df[column_name + '_day'] = df[column_name].dt.day
    return df


date_columns = ["activity_date", "people_date"]  # Use plural "date_columns" instead of "date_column"
for col in date_columns:
    new_df[col] = pd.to_datetime(new_df[col])
    
for col in date_columns:
    extract_year_month_day(new_df, col)
    
new_df = new_df.drop(date_columns, axis=1)
    
new_df.head()

,people_id,activity_id,activity_category,outcome,char_1,group_1,char_2,char_3,char_4,char_5,...,char_35,char_36,char_37,char_38,activity_date_year,activity_date_month,activity_date_day,people_date_year,people_date_month,people_date_day
0,ppl_100,act2_1734928,type 4,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,True,True,False,36,2023,8,26,2021,6,29
1,ppl_100,act2_2434093,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,True,True,False,36,2022,9,27,2021,6,29
2,ppl_100,act2_3404049,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,True,True,False,36,2022,9,27,2021,6,29
3,ppl_100,act2_3651215,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,True,True,False,36,2023,8,4,2021,6,29
4,ppl_100,act2_4109017,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,True,True,False,36,2023,8,26,2021,6,29


In [22]:
COMPRESSED_FILE_PATH = "../data/merged/new_df.pbz2"

if not os.path.exists(COMPRESSED_FILE_PATH):
    f = bz2.BZ2File(COMPRESSED_FILE_PATH, "w")
    pickle.dump(new_df, f)

In [23]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2197291 entries, 0 to 2197290
Data columns (total 49 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   people_id            object
 1   activity_id          object
 2   activity_category    object
 3   outcome              int64 
 4   char_1               object
 5   group_1              object
 6   char_2               object
 7   char_3               object
 8   char_4               object
 9   char_5               object
 10  char_6               object
 11  char_7               object
 12  char_8               object
 13  char_9               object
 14  char_10              bool  
 15  char_11              bool  
 16  char_12              bool  
 17  char_13              bool  
 18  char_14              bool  
 19  char_15              bool  
 20  char_16              bool  
 21  char_17              bool  
 22  char_18              bool  
 23  char_19              bool  
 24  char_20              boo

### <font color="green">Label & One Hot Encoding</font>

In [24]:
new_df.head()

,people_id,activity_id,activity_category,outcome,char_1,group_1,char_2,char_3,char_4,char_5,...,char_35,char_36,char_37,char_38,activity_date_year,activity_date_month,activity_date_day,people_date_year,people_date_month,people_date_day
0,ppl_100,act2_1734928,type 4,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,True,True,False,36,2023,8,26,2021,6,29
1,ppl_100,act2_2434093,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,True,True,False,36,2022,9,27,2021,6,29
2,ppl_100,act2_3404049,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,True,True,False,36,2022,9,27,2021,6,29
3,ppl_100,act2_3651215,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,True,True,False,36,2023,8,4,2021,6,29
4,ppl_100,act2_4109017,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,True,True,False,36,2023,8,26,2021,6,29


In [25]:
columns_to_encode = new_df.select_dtypes(exclude=bool).columns
new_df[columns_to_encode].head()

,people_id,activity_id,activity_category,outcome,char_1,group_1,char_2,char_3,char_4,char_5,...,char_7,char_8,char_9,char_38,activity_date_year,activity_date_month,activity_date_day,people_date_year,people_date_month,people_date_day
0,ppl_100,act2_1734928,type 4,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,type 11,type 2,type 2,36,2023,8,26,2021,6,29
1,ppl_100,act2_2434093,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,type 11,type 2,type 2,36,2022,9,27,2021,6,29
2,ppl_100,act2_3404049,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,type 11,type 2,type 2,36,2022,9,27,2021,6,29
3,ppl_100,act2_3651215,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,type 11,type 2,type 2,36,2023,8,4,2021,6,29
4,ppl_100,act2_4109017,type 2,0,type 2,group 17304,type 2,type 5,type 5,type 5,...,type 11,type 2,type 2,36,2023,8,26,2021,6,29
